In [ ]:
#ollama run brxce/monadgpt modern french
#ollama

In [1]:
!ls data/

liv0232_ideedelaperfectiondelape.txt


In [6]:
!pip install alto2txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [13]:
!ls

LIV0144_altos_transcribed  LIV0304_altos_transcribed  venv
LIV0227_altos_transcribed  output
LIV0299_altos_transcribed  RAG_historical.ipynb


In [9]:
import os
import re
import xml.etree.ElementTree as ET

def convert_alto_to_text(input_folder, output_file):
    # Define ALTO namespace
    ns = {'alto': 'http://www.loc.gov/standards/alto/ns-v4#'}
    
    # Get all XML files and sort them numerically
    xml_files = [f for f in os.listdir(input_folder) if f.endswith('.xml')]
    xml_files.sort(key=lambda x: int(re.findall(r'\d+', x)[0]))
    
    with open(output_file, 'w', encoding='utf-8') as txt_file:
        for xml_file in xml_files:
            xml_path = os.path.join(input_folder, xml_file)
            
            # Parse the XML
            tree = ET.parse(xml_path)
            root = tree.getroot()
            
            # Find the MainZone TYPE ID
            mainzone_id = None
            for tag in root.findall(".//alto:OtherTag", ns):
                if tag.attrib.get('LABEL') == 'MainZone':
                    mainzone_id = tag.attrib.get('ID')
                    break
            
            if mainzone_id:
                # Find all TextBlocks with the MainZone ID
                main_blocks = root.findall(f".//alto:TextBlock[@TAGREFS='{mainzone_id}']", ns)
                
                text_content = []
                for block in main_blocks:
                    for line in block.findall(".//alto:TextLine", ns):
                        line_text = []
                        for string in line.findall(".//alto:String", ns):
                            if 'CONTENT' in string.attrib:
                                line_text.append(string.attrib['CONTENT'])
                        if line_text:
                            text_content.append(' '.join(line_text))
                
                # Write the text with proper line breaks
                if text_content:
                    txt_file.write('\n'.join(text_content) + '\n\n')
            
            print(f"Processed: {xml_file}")
    
    print(f"All text written to {output_file}")

In [10]:
# Example usage
input_folder = 'LIV0227_altos_transcribed'
output_folder = 'LIV0227_altos_transcribed.txt'
convert_alto_to_text(input_folder, output_folder)

Processed: f0-plat-superieur.xml
Processed: f1-contreplat-sup.xml
Processed: f2-page-de-garde-recto.xml
Processed: f3-page-de-garde-verso.xml
Processed: f4-np.xml
Processed: f5-np.xml
Processed: f6-1r.xml
Processed: f7-1v.xml
Processed: f8-2r.xml
Processed: f9-2v.xml
Processed: f10-3r.xml
Processed: f11-3v.xml
Processed: f12-4r.xml
Processed: f13-4v.xml
Processed: f14-5r.xml
Processed: f15-5v.xml
Processed: f16-6r.xml
Processed: f17-6v.xml
Processed: f18-7r.xml
Processed: f19-7v.xml
Processed: f20-8r.xml
Processed: f21-8v.xml
Processed: f22-9r.xml
Processed: f23-9v.xml
Processed: f24-10r.xml
Processed: f25-10v.xml
Processed: f26-11r.xml
Processed: f27-11v.xml
Processed: f28-12r.xml
Processed: f29-12v.xml
Processed: f30-13r.xml
Processed: f31-13v.xml
Processed: f32-14r.xml
Processed: f33-14v.xml
Processed: f34-15r.xml
Processed: f35-15v.xml
Processed: f36-16r.xml
Processed: f37-16v.xml
Processed: f38-17r.xml
Processed: f39-17v.xml
Processed: f40-18r.xml
Processed: f41-18v.xml
Processed:

# Normalisation
`!git clone https://github.com/rbawden/ModFr-Norm.git`

In [2]:
!pip install torch transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.9 MB/s eta 0:00:00
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9

In [11]:
import re

def regroup_lines_into_sentences(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            sentences = []
            current_sentence = ""

            for line in lines:
                line = line.strip()  # Supprimer les espaces en début et fin de ligne
                if line:  # Vérifier si la ligne n'est pas vide
                    # Ajouter la ligne à la phrase courante
                    current_sentence += line + " "
                    if re.search(r'[.!?]$', line):  # Vérifier si la ligne se termine par un marqueur de fin de phrase
                        sentences.append(current_sentence.strip())  # Ajouter la phrase complète à la liste
                        current_sentence = ""  # Réinitialiser la phrase courante
                    current_sentence = current_sentence.replace('¬ ', '')  # Remplacer '¬ ' par rien
            return sentences
    except FileNotFoundError:
        return "Fichier non trouvé."

file_path = output_folder  # Remplacez ceci par le chemin de votre fichier .txt
sentences = regroup_lines_into_sentences(file_path)

In [12]:
from tqdm.notebook import tqdm

In [13]:
from transformers import pipeline
normaliser = pipeline(model="rbawden/modern_french_normalisation", batch_size=32, beam_size=5, cache_file="./cache.pickle", device=0, trust_remote_code=True)


with open("normalized.txt", "w", encoding="utf-8") as output_file:
    # Wrap sentences with tqdm and store total length
    for line in tqdm(sentences, desc="Normalizing text"):
        try:
            normalized_line = normaliser([line])
            output_file.write(normalized_line[0]['text'].strip() + "\n")
        except Exception as e:
            print(f"Error processing line: {line}")
            print(f"Error details: {e}")
            output_file.write(line.strip() + "\n")
            continue

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at rbawden/modern_french_normalisation and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Normalizing text:   0%|          | 0/151 [00:00<?, ?it/s]

Error processing line: VRCE que le Sieur lean Couſin voulant faire les demonſtrations neceſY ſaires à l’inſtruction & parfaite intelligence de l’Art de Portraicture, il a eſté obligé de parler des lignes les plus ordinaires de la Geometrie, qui ſont la Perpendiculaire, Diagonale, Parallele, Orthogonale, & autres; & de leurs interſections.
Error details: list index out of range
Error processing line: A ....EIAISPABA.PAVY& .M ..22 Q ..- - . . so. e..
Error details: list index out of range
Error processing line: JOVR racourcir les mains, faut tirer lignes perpendiculaires punctuées des Y mains veuës de coſté ou profil, ma rquées de leurs proportions & meſures 1.2.3.
Error details: list index out of range
Error processing line: A u A B a Ain IN 2 vupe eu Palim e 2 2 18 2L F Lii 14 – PARTICVLARITEZ DV CORPS HVMAIN, TANT QV HEVANTQVE UV DERRIERE, DENET LES ESPAVEEA iuſques aux genitoires, & proportion d’iceluy.
Error details: list index out of range
Error processing line: I u un vn me

# Analyse document
https://github.com/langchain-ai/langchain/blob/master/cookbook/analyze_document.ipynb

https://python.langchain.com/docs/use_cases/summarization/

In [ ]:
!pip install langchain

In [28]:
with open("data/normalized.txt") as f:
    state_of_the_union = f.read()

In [29]:
from langchain.chains import AnalyzeDocumentChain
from langchain_community.llms import Ollama

llm = Ollama(model="mixtral:8x7b")

In [30]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm, chain_type="map_reduce")

In [31]:
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

In [32]:
qa_document_chain.run(
    input_document=state_of_the_union,
    question="What does the author think about estampes?",
)

" The author does not express a specific opinion about estampes in general, but they do mention that some prints or academic studies must be adjusted according to perspective and the original viewpoint. They also praise a particular engraving for its merit, grandeur, rare considerations, and admirable expression. Additionally, they criticize another engraving for lacking force in expressing violent passions. However, these comments are specific to individual works rather than estampes as a whole. Therefore, it is not possible to determine the author's overall opinion about estampes based on this document alone."

In [33]:
qa_document_chain.run(
    input_document=state_of_the_union,
    question="Summarizes the document and the author's thinking",
)

' The document is a book written by Roland Freart Sieur de Chambray titled "IDE DE DE PERFECTION DE LA PEINTURE" and published in Le Mans by IA CQVESYs AMBART. The book aims to demonstrate the principles of art and painting through examples, comparing ancient painters like Leonardo da Vinci, Raphael, Julius Roman, and Poussin with some modern French painters. The author emphasizes the importance of perspective in painting and provides axioms for it. The document also highlights the significance of accurately depicting Costumes to enhance the quality of a painter\'s work and avoid criticism.'

In [34]:
qa_document_chain.run(
    input_document=state_of_the_union,
    question="Give me a list of all the individuals cited by the author in these arguments",
)

' The president did not mention Michael Jackson.\n\nThe individuals cited by the author in these arguments are:\n\n1. Géomettres (geometers)\n2. Peintres (painters)\n3. Es (artists or people who make engravings)\n4. Madame (a person to whom the document is addressed)\n5. Raphael\n6. Marc-Antoine\n7. The women in the engraving\n8. The bourreaux (executioners) in the engraving\n9. The assassins of "ces urs petits In-ô"\n10. The malheureuses mères (unhappy mothers) of the children being killed\n11. The Peintre (the Painter), who is Raphael.\n12. NÔTRE SEIGNEUR (Jesus Christ)\n13. La Vierge (Virgin Mary)\n14. Les Maries (the Marys, plural form of Marie in French, likely referring to Mary Magdalene and other female followers of Jesus)\n15. Joseph JRimathie (Joseph of Arimathea)\n16. Nicodeme (Nicodemus)\n17. Saint Jean (John the Apostle)\n18. Ioseph d\'Arimathie (Joseph of Arimathea, mentioned again)\n19. La PEINTVRE (the Painter, generic term, not an individual)\n20. The author himself\n21

In [ ]:
# citation -> https://python.langchain.com/docs/use_cases/question_answering/citations/

# RAG 
https://github.com/langchain-ai/langchain/blob/master/cookbook/Semi_structured_multi_modal_RAG_LLaMA2.ipynb